<h1>EXPLORING & CLUSTERING NEIGHBORHOODS OF TORONTO<h1>

<h2>PART 1 OF THE FINAL SUBMISSION<h2>

In [1]:
#IMPORT NECCESSARY LIBRARIES
import pandas as pd
import numpy as np
import requests
import xml

print('Libraries imported')

Libraries imported


In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
from bs4 import BeautifulSoup

In [4]:
#Scraping data from wikipedia
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bsoup = BeautifulSoup(url, 'html.parser')

In [5]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [6]:
# Append the data into respective lists
for row in bsoup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [7]:
#Createa a new pandas dataframe from the list
df_toronto=pd.DataFrame({"POSTALCODE" :postalCodeList,"Borough":boroughList, "Neighborhood": neighborhoodList})
df_toronto.head()

,POSTALCODE,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [8]:
#cleaning the \n with ' '

df_toronto = df_toronto.replace('\n',' ', regex=True)
df_toronto.head()

,POSTALCODE,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [18]:
#IGNORING THE CELLS THAT HAVE N/A VALUES IN BOROUGH COLUMN
df_toronto['Borough'].replace('Not assigned ', np.nan, inplace=True)
df_toronto.dropna(subset=['Borough'], inplace=True)
df_toronto.head()


,POSTALCODE,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [20]:
#Group neighborhoods in the same borough
df_toronto = df_toronto.groupby(['POSTALCODE', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df_toronto.columns = ['POSTALCODE','Borough','Neighborhood']
df_toronto

,POSTALCODE,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [22]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_toronto.loc[df_toronto['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_toronto['Borough']
df_toronto


,POSTALCODE,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
